In [1]:
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
tqdm.pandas()

bho_cands = pd.read_pickle("bho_queries_britwikidata_candidates+ids_wikigaz_en_002.pkl")
#bho_cands = bho_cands.head(100)

britwikidata_gazetteer = pd.read_pickle("/home/mcollardanuy/PlaceLinking/toponym_matching/gazetteers/britwikidata_gazetteer.pkl")
britwikidata_gazetteer.head()

,wkid,altname,source,lat,lon
0,Q63607865,Kilberry Head,english_label,55.818446,-5.667253
1,Q63607865,Kilberry,geonames,55.818446,-5.667253
2,Q63759247,Sandhurst Memorial Park,english_label,51.344000,-0.793000
3,Q63781193,Currie railway station,english_label,55.903100,-3.283500
4,Q63859929,Stannington War Memorial,english_label,53.392659,-1.548018


In [2]:
from wikimapper import WikiMapper
from pathlib import Path
import urllib.parse

mapper = WikiMapper("/resources/wikidata2wikipedia/index_enwiki-20190420.db")

def map_wikidata2wikidump(wikidataId):
    path = "/resources/wikipedia/extractedResources/Aspects/"
    wikititles = mapper.id_to_titles(wikidataId)
    wikititles = [urllib.parse.quote(title.replace("_"," ")) for title in wikititles]
    wikidumps = [path + title+".json" for title in wikititles if Path(path + title+".json").is_file()]
    return wikidumps


single_best_match = 0
single_best_wikipedia_match = 0
single_possible_match = 0
multiple_best_match = 0
multiple_possible_match = 0
multiple_best_but_no_wiki_match = 0
no_wiki_match = 0

cutoff = 5

for index, row in bho_cands.iterrows():
    wikidata_cands = row["wikidata_cands"]
    best_match = [x for x,y in wikidata_cands.items() if y==0.0]
    if len(best_match)>0:
        if len(best_match)==1:
            single_best_match+=1
        else:
            wikipages = [[wikidataId,page] for wikidataId in best_match for page in map_wikidata2wikidump(wikidataId)]
            if len(wikipages)==1:
                single_best_wikipedia_match+=1
            elif len(wikipages)>1:
                multiple_best_match+=1
            elif len(wikipages)==0:
                multiple_best_but_no_wiki_match+=1
    else:
        selected = [x for x in wikidata_cands.keys()][:cutoff]
        wikipages = [[wikidataId,page] for wikidataId in selected for page in map_wikidata2wikidump(wikidataId)]
        if len(wikipages)==1:
            single_possible_match+=1
        elif len(wikipages)>1:
            multiple_possible_match+=1
        elif len(wikipages)==0:
            no_wiki_match+=1



print ("perfect unique match:",round(single_best_match/len(bho_cands),3))
print ("perfect unique wikipedia match:",round(single_best_wikipedia_match/len(bho_cands),3))
print ("multiple best candidates:",round(multiple_best_match/len(bho_cands),3))
print ("multiple best candidates but no wikipedia match:",round(multiple_best_but_no_wiki_match/len(bho_cands),3))
print ("single possible candidate:",round(single_possible_match/len(bho_cands),3))
print ("multiple possible candidates:",round(multiple_possible_match/len(bho_cands),3))
#print ("no single best candidate:",(len(bho_cands)-single_match-multiple_match-no_wiki_match)/len(bho_cands))
print ("no wiki page available",round(no_wiki_match/len(bho_cands),3))



perfect unique match: 0.464
perfect unique wikipedia match: 0.05
multiple best candidates: 0.12
multiple best candidates but no wikipedia match: 0.004
single possible candidate: 0.053
multiple possible candidates: 0.288
no wiki page available 0.021
